In [51]:
from bs4 import BeautifulSoup as bs
import requests
import json
import re



In [52]:
r = requests.get('https://www.yahoo.com')
r.raise_for_status()

Returns the content of url 

In [53]:
html = bs(r.content)

In [54]:
# print(html.prettify())

This is where the the each article are stored in the html code

In [55]:
body = html.find_all(class_="List(n) P(0) grid-layout stream-grid stream-items")
content = body[0].find_all(class_="Pos(r) D(f)")


Code below get the gets and stores the title of each articles and the link to the article and appends it to a list

In [56]:
info = []
for i in content:
    try:
        i.get_text('}').split('}')[4]
    except:
        continue
    else:
        info.append({i.get_text('}').split('}')[4]:i.find('a').get('href')})
     

    

# with open('News_articles2.json','w') as f:
#     json.dump(info,f)

Check if the new news article is not in the json file, then append it to the json file then save it

num_of_new_articles is going to be useful later on when you automate the code to only send the newly added articles


In [57]:


with open('News_articles2.json') as f:
    js_file = json.load(f)

num_of_new_articles = 0
for i in info:
    if i not in js_file:
        num_of_new_articles = num_of_new_articles + 1
        print(i)
        js_file.append(i)
           
with open('News_articles2.json','w') as f:
    json.dump(js_file,f)
        


{'Dow inches up as inflation trends stay positive: Stock market news today': 'https://finance.yahoo.com/news/dow-inches-up-as-inflation-trends-stay-positive-stock-market-news-today-173535572.html'}
{"San Francisco Fed’s Daly: 'There’s still more work to do' on inflation": 'https://finance.yahoo.com/news/san-francisco-feds-daly-theres-still-more-work-to-do-on-inflation-172924823.html'}
{"Social Security's cost-of-living increase could be only 3%: Will that be enough?": 'https://finance.yahoo.com/news/social-securitys-cost-of-living-increase-could-be-only-3-will-that-be-enough-183700266.html'}
{"Lil Tay not actually dead: Child rapper and her brother are 'safe and alive' after apparent death hoax": 'https://www.yahoo.com/entertainment/lil-tay-not-dead-child-rapper-apparent-death-hoax-185916914.html'}
{"July inflation data shows 'convincing' signs price pressures easing, taking heat off Fed": 'https://finance.yahoo.com/news/july-inflation-data-shows-convincing-signs-price-pressures-easing

In [58]:
# load the update json file 
with open('News_articles2.json') as f:
    js_daily = json.load(f)
    
working_set = js_daily


In [59]:
len(working_set)

1003

Remove the ads articles

I noticed the ads articles don't have yahoo in the url or the keys are just whitespace

I want to loop through each dictionary and use regex to find the dictionary that does have '.yahoo' and delete them from the list because does are ads or atleat i know they are not news article

Remove instances with whitespace as keys in the dictionary

Use regex to check for .yahoo in the dictionaty value

we reversed the the working_set, so it goes from most recent article to least recent

In [60]:
# remove the ads articles
# i noticed the ads articles don't have yahoo in the url or the keys are just whitespace

# i want to loop through each dictionary and use regex to find the dictionary that
# does have '.yahoo' and delete them from the list because does are ads or atleat i know they are not news article

# remove instances with whitespace as keys in the dictionary
# use regex to check for .yahoo in the dictionaty value

working_set_reversed = working_set[::-1]
for j,i in enumerate(working_set_reversed[:num_of_new_articles]):
    if  ' ' in i or not re.search('\.yahoo\.com',working_set_reversed[j][list(i.keys())[0]]):
        num_of_new_articles = num_of_new_articles - 1
        
remove_item = []
for j,i in enumerate(working_set_reversed):
    if  ' ' in i or not re.search('\.yahoo\.com',working_set_reversed[j][list(i.keys())[0]]):
        remove_item.append(working_set_reversed[j])

for i in remove_item:
    working_set_reversed.remove(i)    
    



In [61]:
# for j,i in enumerate(working_set_reversed[:num_of_new_articles]):
#     print(i.keys())
#           [list(i.keys())[0]])

In [62]:
for j,i in enumerate(working_set_reversed[:num_of_new_articles]):

    print(working_set_reversed[j][list(i.keys())[0]])
print('\n')    
print("we have "+str(len(working_set))+" articles in total and we have "+str(num_of_new_articles)+" newly found articles")

https://www.yahoo.com/entertainment/mom-called-jury-duty-no-144010166.html
https://www.yahoo.com/entertainment/clint-eastwood-reaction-former-mistress-160040129.html
https://www.yahoo.com/entertainment/lil-tay-emerges-she-not-174410870.html
https://www.yahoo.com/lifestyle/21-heartbreaking-images-hawaiis-deadly-181620956.html
https://news.yahoo.com/kristin-harila-accused-walking-over-162436888.html
https://www.yahoo.com/lifestyle/paige-spiranac-hits-golf-course-171828820.html
https://finance.yahoo.com/news/americas-highest-earners-are-getting-hit-hardest-by-a-slowing-labor-market-190627662.html
https://news.yahoo.com/student-cleared-in-stolen-airpods-case-after-3-year-ticket-battle-192335863.html
https://www.yahoo.com/lifestyle/helen-mirren-castor-oil-130034012.html
https://sports.yahoo.com/why-it-wont-just-be-up-to-the-james-family-whether-bronny-plays-for-usc--or-any-other-school-150722851.html
https://www.yahoo.com/lifestyle/after-multiple-scandals-beauty-influencer-jaclyn-hill-annou

We go to every dictionary values, and use request to get the content of each articles then we use bs4 to get the p tag, which is the text of the article

since we only want the content of the article but the p tag returns some other information after the end of the article, say some recommendation to view other article and we dont want that, one thing i noticed is that most article end with some words which i store in the end_words list, so i loop through each article and once i see any any word in the end_words list and delete from that word to the end. I wouldn't say its 100 percent accurate but it's like 90 percent accurate cause they are just so many possible ways for an article to end

In [63]:
# we go every dictionary values, load it use request and get the <p> tag
end_words= ['Follow us on Twitter and Instagram',
"follow us on Facebook, Twitter, and Instagram",'More Latest News!','TRENDING',
'Recommended Stories',
'TRENDING','POPULAR', 'Read More','Latest Stories']

# end_words = [end_word.lower() for end_word in end_words]

for index,working in enumerate(working_set_reversed[:num_of_new_articles]):
    res = requests.get(list(working.values())[0])
    soup = bs(res.content)
    text = [i.get_text() for i in soup.find_all(['h2','p'])]

    for word in text:
        if word in end_words:
            print(index,word)
            working_set_reversed[index][list(working_set_reversed[index].keys())[0]]  = text[0:text.index(word)]
            break
        else:
            working_set_reversed[index][list(working_set_reversed[index].keys())[0]]  = text[0:]
                

0 TRENDING
1 TRENDING
2 TRENDING
3 TRENDING
4 Recommended Stories
5 TRENDING
6 TRENDING
7 Recommended Stories
8 TRENDING
9 Recommended Stories
10 TRENDING
11 TRENDING
12 Recommended Stories
13 Recommended Stories
14 Recommended Stories
15 TRENDING
16 TRENDING
17 Recommended Stories
18 TRENDING
19 Recommended Stories
20 TRENDING
21 Recommended Stories
22 TRENDING
23 Recommended Stories
24 Recommended Stories
25 Recommended Stories
27 Recommended Stories
28 TRENDING
29 Recommended Stories
30 TRENDING
31 TRENDING
32 TRENDING
33 Recommended Stories
34 Recommended Stories
35 TRENDING
36 TRENDING
37 TRENDING
38 TRENDING
39 TRENDING


In [64]:
# working_set_copy = working_set[:]

In [65]:
# working_set_copy[:15]

In [66]:
# working_set_copy

# Cleaning the articles

These are some things that need to be cleaned from the articles

In [67]:
# pic.twitter.com/8wwe2lximt
# — theblaze (@theblaze) july 27, 2023
# — rep. nancy mace (@repnancymace) july 27, 2023
# more from 
# \xa0
# \'
# [...]

# —
# . that heel flick backwards from @_aryborges 🔥pic.twitter.com/agetnxr7ln — national women’s soccer league (@nwsl) 
# july 24, 2023 she earned her hat 
# trick 20 minutes of play later on another header that went between the legs of panama keeper yenith bailey. she's
# becoming inevitable 🇧🇷hat trick ary borges 🎩 pic.twitter.com/fygalerheo — fox soccer (@foxsoccer) july 24, 2023 
# marta came on for borges in the 75th minute. 

# . germany held nothing back in this one 😤🇩🇪watch all six goals from germany's 2023 fifa women's world cup opener ⬇️ pic.twitter.com/bbx7nlq8vb — fox soccer (@foxsoccer) july 24, 2023 the six goals are the most of the tournament so far. 
# https://t.co/hr1vvd7bmw — tom pelissero (@tompelissero) july 24, 2023 this is a brutal turn of events for the 26-year-old hines.

# . he lost $1.67m salary due to buyout.rep'd by pat brisson @caahockey https://t.co/8w0mfcfrau — puckpedia (@puckpedia) july 24, 2023

# \xa0
# remove \n and [it]
# ”


In [68]:
for j,i in enumerate(working_set_reversed[:num_of_new_articles+1]):
    working_set_reversed[j][list(working_set_reversed[j].keys())[0]] = " ".join(working_set_reversed[j][list(working_set_reversed[j].keys())[0]])

In [69]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "\U0001F700-\U0001F77F"  # alchemical symbols
                           "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           "\U0001FA00-\U0001FA6F"  # Chess Symbols
                           "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           "\U00002702-\U000027B0"  # Dingbats
                           "\U000024C2-\U0001F251"
                           "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "\U0001F300-\U0001F5FF"  # symbols & pictographs
                           "\U0001F600-\U0001F64F"  # emoticons
                           "\U0001F680-\U0001F6FF"  # transport & map symbols
                           "\U0001F700-\U0001F77F"  # alchemical symbols
                           "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"
    u"\U0001F600-\U0001F64F"  # emoticons
                           
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                           "]+", flags=re.UNICODE)

def clean_up(working_set_copy):
    for j,i in enumerate(working_set_copy[:num_of_new_articles]):
        working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub('"|“', "", working_set_copy[j][list(working_set_copy[j].keys())[0]])
        working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub('\\\xa0', " ", working_set_copy[j][list(working_set_copy[j].keys())[0]])

        working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub("\\\'","'",working_set_copy[j][list(working_set_copy[j].keys())[0]])
    
        if re.search(emoji_pattern,working_set_copy[j][list(working_set_copy[j].keys())[0]]):
            working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub(emoji_pattern,'',working_set_copy[j][list(working_set_copy[j].keys())[0]])

            reg = re.search(r'(\.|,)([ ]){2,}(\.|,)',working_set_copy[j][list(working_set_copy[j].keys())[0]])
            reg2 = re.search(r'\s{2,}(\.|,)',working_set_copy[j][list(working_set_copy[j].keys())[0]])
            if reg:
                working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub(r'(\.|,)([ ]){2,}(\.|,)',reg.groups()[-1]+' ',working_set_copy[j][list(working_set_copy[j].keys())[0]])
            if reg2:
                working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub(r'\s{2,}(\.|,)',reg2.groups()[-1]+' ',working_set_copy[j][list(working_set_copy[j].keys())[0]])

        reggs = re.search('(\[)(.+)(\])',working_set_copy[j][list(working_set_copy[j].keys())[0]])
        if reggs:
            working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub('(\[)(.+)(\])',reggs.groups()[1],working_set_copy[j][list(working_set_copy[j].keys())[0]])
            
        working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub('([\. | |pic\.].{,25}[ ]+—[ ]+[a-zA-Z ]+\s\(@\w+\)\s\w{3,9}\s\d{1,2},\s\d{2,4})','. ',working_set_copy[j][list(working_set_copy[j].keys())[0]])
        working_set_copy[j][list(working_set_copy[j].keys())[0]] = re.sub(r'\s{2,}View this post on Instagram\s{2,}A post shared by .{2,15}\s?\(@.+\)','',working_set_copy[j][list(working_set_copy[j].keys())[0]])
    return working_set_copy[:num_of_new_articles]
        
    

    
articles = clean_up(working_set_reversed)
print(articles[:5])

[{'Mom Called for Jury Duty Has No Babysitter — So She Shows Up with 1-Year-Old Triplets in Tow (Exclusive)': "Torrey Scow didn't want to face the $1000 fine for skipping jury duty, so she packed up her four kids for what ended up being an adventure Torrey Scow Instagram Torrey Scow is used to getting looks as she goes about her daily life with four kids — triplets Lexi, Zoey and Lincoln, 18 months, and daughter Emory, 5 — but when she shared her experience at jury duty, others really took notice. In June, the mom of four was called to jury duty and thought a lack of child care would allow her to reschedule. I even sent them a picture because I wanted them to know how hard it is for me to find a babysitter, because the only person I have that can handle all four kids is either myself or my husband, or my mom, Torrey tells PEOPLE of the incident. It'd be easier if they were older toddlers, but they're still babies who need bottles and changing at different times. But they ended up sendi

# nltk 

We basically just take the most common word from each articles and that's our summary 

In [70]:
from collections import Counter
from nltk.tokenize import sent_tokenize,word_tokenize,PunktSentenceTokenizer
summary = []

for j,i in enumerate(working_set_reversed[:num_of_new_articles]):
    counts = Counter(sent_tokenize(working_set_reversed[j][list(working_set_reversed[j].keys())[0]])).most_common(int(len(sent_tokenize(working_set_reversed[j][list(working_set_reversed[j].keys())[0]]))*0.3))

    summary.append({list(working_set_reversed[j].keys())[0]:' '.join([k[0] for k in counts])})


In [71]:
with open('summariesconfirm.json') as f:
    summary_file = json.load(f)


for i in summary:
    if i not in summary_file :
        print(i)
        summary_file.append(i)
           
with open('summariesconfirm.json','w') as f:
    json.dump(summary_file,f)


{'Mom Called for Jury Duty Has No Babysitter — So She Shows Up with 1-Year-Old Triplets in Tow (Exclusive)': "Torrey Scow didn't want to face the $1000 fine for skipping jury duty, so she packed up her four kids for what ended up being an adventure Torrey Scow Instagram Torrey Scow is used to getting looks as she goes about her daily life with four kids — triplets Lexi, Zoey and Lincoln, 18 months, and daughter Emory, 5 — but when she shared her experience at jury duty, others really took notice. In June, the mom of four was called to jury duty and thought a lack of child care would allow her to reschedule. I even sent them a picture because I wanted them to know how hard it is for me to find a babysitter, because the only person I have that can handle all four kids is either myself or my husband, or my mom, Torrey tells PEOPLE of the incident. It'd be easier if they were older toddlers, but they're still babies who need bottles and changing at different times. But they ended up sendin

In [2]:
import os

In [26]:
os.environ

environ{'__CFBundleIdentifier': 'com.apple.Terminal',
        'TMPDIR': '/var/folders/nq/qb822jkx38j2c_4_zwzhvxf80000gn/T/',
        'XPC_FLAGS': '0x0',
        'LaunchInstanceID': 'E64CFA70-CA1D-45BE-9A2F-52A48B2F2074',
        'TERM': 'xterm-color',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.W4P8RLZe6Z/Listeners',
        'SECURITYSESSIONID': '18c8d',
        'XPC_SERVICE_NAME': '0',
        'TERM_PROGRAM': 'Apple_Terminal',
        'TERM_PROGRAM_VERSION': '447',
        'TERM_SESSION_ID': '052C202F-7E58-4106-B580-8DB7A2DD6855',
        'SHELL': '/bin/zsh',
        'HOME': '/Users/adeoyedipo',
        'LOGNAME': 'adeoyedipo',
        'USER': 'adeoyedipo',
        'PATH': '/Users/adeoyedipo/opt/anaconda3/bin:/Users/adeoyedipo/opt/anaconda3/condabin:/Library/Frameworks/Python.framework/Versions/3.6/bin:/Library/Frameworks/Python.framework/Versions/3.9/bin:/Library/Frameworks/Python.framework/Versions/3.8/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/

In [ ]:
# CLIENT_ID=8879a6eab24f4834bc2b132380e37127
# CLIENT_SECRET=46e697503e984208be7141da13b67b11